 # Portfolio Optimization using cvxpy
 # Minimize portfolio variance and track index

In [3]:
import cvxpy as cvx
import numpy as np

$  Minimize \left [ \sigma^2_p + \lambda \sqrt{\sum_{1}^{m}(weight_i - indexWeight_i)^2} \right  ]$ 


$\mathbf{x} = \begin{bmatrix}
x_1 &...& x_M
\end{bmatrix}
$

covariance matrix $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\
\end{bmatrix}$


portfolio variance $\sigma^2_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

We want to minimize the distance between index weights and our weights(index is weighted).
 I will use L2 norm as distance calculation: $\sqrt{\sum_{1}^{n}(weight_i - indexWeight_i)^2}$  = $\left \| \mathbf{x} - \mathbf{index} \right \|_2$.  
cvxpy has a function called [norm()] which I will use to declare the L2 norm.

### constraints
[x >= 0, sum(x) == 1], no shorting. long only.

In [4]:
import cvxpy as cvx
import numpy as np

def optimize_portfolio(returns, index_weights, scale=.00001):
    """   
    Input
    ----------
    returns : numpy.ndarray where each row is stock series(2D).
        
    index_weights : numpy.ndarray containing weights of the index(1D).
        
    scale : float, lambda aka scaling factor
    
    Output:
    -------
    array of optimal weights
    """
    # number of stocks m is number of rows of returns, and also number of index weights
    m = len(returns) 
    #covar mat of returns
    cov = np.cov(returns) 
    # x var
    x =  cvx.Variable(m)
    #portfolio variance
    portfolio_variance = cvx.quad_form(x,cov)
    # L2 norm between portfolio and index weights
    distance_to_index = cvx.norm(x-index_weights,2)#L2 norm
    #constraints
    constraints = [sum(x)==1,x>=0] 
    #objective
    obj = portfolio_variance + scale*distance_to_index
    objective = cvx.Minimize(obj) 
    problem = cvx.Problem(objective,constraints)
    minv = problem.solve()
    
    x_values = x.value
    
    return x_values



In [7]:
days_per_year = 252
years = 10
total_days = days_per_year * years

return_market = np.random.normal(loc=0.05, scale=0.3, size=days_per_year)
return_1 = np.random.uniform(low=-0.000001, high=.000001, size=days_per_year) + return_market
return_2 = np.random.uniform(low=-0.000001, high=.000001, size=days_per_year) + return_market
return_3 = np.random.uniform(low=-0.000001, high=.000001, size=days_per_year) + return_market
returns = np.array([return_1, return_2, return_3])

index_weights = np.array([0.9,0.15,0.05])
x_values = optimize_portfolio(returns, index_weights, scale=.00001)

print("optimized weights: {}".format(x_values))

optimized weights: [0.86666278 0.11658134 0.01675587]
